In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from datetime import timedelta

In [2]:
import pandas as pd
import numpy as np

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [4]:
url='https://github.com/hamzahshabbir96/Temporal-forecasting-of-temperature-and-Carbon-Monoxide-CO-sensor-data-one-day-ahead/blob/main/taxi.csv'

In [9]:
from google.colab import files
uploaded = files.upload()

Saving taxi.csv to taxi.csv


In [10]:
file=pd.read_csv('taxi.csv')

In [7]:
file.head()

,DS,Y
0,01/03/2018 00:00:00,124
1,01/03/2018 01:00:00,85
2,01/03/2018 02:00:00,71
3,01/03/2018 03:00:00,66
4,01/03/2018 04:00:00,43


In [11]:
df=file.copy()
df['datetime']=df['datetime'].apply(lambda x: str(x[0:-3])+':00:00')
df_grouped=df.groupby('datetime')
df_grouped['num_orders'].agg(np.sum)
df_new=pd.DataFrame(df_grouped['num_orders'].agg(np.sum))

In [76]:
train=df_new[:3976]
test=df_new[3977:]
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
scaler.fit(train)
scaled_train=scaler.transform(train)
scaled_test=scaler.transform(test)

In [106]:
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import Dropout


In [133]:
inp=12
nf=1

In [134]:
time_generator=TimeseriesGenerator(scaled_train,scaled_train,length=inp,batch_size=64)

In [136]:
model=Sequential()
model.add(LSTM(128,activation='relu',input_shape=(inp,nf),return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(64,activation='relu',return_sequences=False))
#model.add(LSTM(32,activation='relu'))

model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 100)               40800     
                                                                 
 dense_8 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [137]:
model.fit_generator(time_generator,epochs=20)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


62/62 [==============================] - 5s 37ms/step - loss: 0.0121
Epoch 2/20
62/62 [==============================] - 2s 37ms/step - loss: 0.0081
Epoch 3/20
62/62 [==============================] - 2s 37ms/step - loss: 0.0081
Epoch 4/20
62/62 [==============================] - 2s 37ms/step - loss: 0.0078
Epoch 5/20
62/62 [==============================] - 2s 36ms/step - loss: 0.0077
Epoch 6/20
62/62 [==============================] - 2s 36ms/step - loss: 0.0076
Epoch 7/20
62/62 [==============================] - 2s 36ms/step - loss: 0.0075
Epoch 8/20
62/62 [==============================] - 2s 37ms/step - loss: 0.0068
Epoch 9/20
62/62 [==============================] - 2s 36ms/step - loss: 0.0065
Epoch 10/20
62/62 [==============================] - 2s 36ms/step - loss: 0.0062
Epoch 11/20
62/62 [==============================] - 2s 37ms/step - loss: 0.0059
Epoch 12/20
62/62 [==============================] - 2s 36ms/step - loss: 0.0058
Epoch 13/20
62/62 [=============================

In [138]:
predictions=[]
first_eval_batch=scaled_train[-inp:]
curr_batch=first_eval_batch.reshape(1,inp,nf)
for i in range(len(test)):
  curr_pred=model.predict(curr_batch)[0]
  predictions.append(curr_pred)
  curr_batch=np.append(curr_batch[:,1:,:],[[curr_pred]],axis=1)
true_pred=scaler.inverse_transform(predictions)


In [69]:
#predictions

In [92]:
true_pred=scaler.inverse_transform(predictions)

In [70]:
#true_pred

In [71]:
#test

In [130]:
import math
def error_rmse(actual,predicted): 
    MSE = np.square(np.subtract(actual,predicted)).mean() 
    RMSE = math.sqrt(MSE)
    return RMSE

In [139]:
err=error_rmse(test,true_pred)
err

54.79789374982491